In [1]:
#Codigo para clasificar mensajes recibidos + matchear (SOLO CONFIRMACION DE PROPERATI)
#Ultima actualizacion: 6/05/2022

#Inputs: (A) base de scraps de properati (randomization) y (B) base de mails con respuestas, en las cuales haya confirmaciones de properati
#Outputs: (A) sent_mails_conf_envio_[FECHA], que agrupa las publicaciones a las que les enviamos correctamente el mail, y (B) not_sent_mails_conf_envio_[FECHA]_[PAIS] que muestra, para cada país, las inmobiliarias a las que les tenemos que reenviar los correos.

In [2]:
# Importar paquetes necesarios para la clasificacion

import pandas as pd
import numpy as np
from datetime import datetime
import matching_v1 as matching
import tools

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

pd.options.mode.chained_assignment = None  # default='warn'

#### Ronda y fecha de la ronda
ronda = '2'
fecha_out = '2022_05_10'
fecha_rand = '2022_05_03'
fecha_descarga = '2022_05_19'

#### Establecer rutas de entrada y salida para las bases (si se cambian, comentar las que no se usen)
#Rutas Nico
path_dropbox = r"C:\Users\Nico\Dropbox"

#Rutas Juli
# path_data = r"C:\Users\Nico\Dropbox\BID-LGBTQ+\campo"

#Rutas Luis
# path_data = r"C:\Users\Nico\Dropbox\BID-LGBTQ+\campo"

#### Rutas utilizadas:
path_data = path_dropbox + r"\BID-LGBTQ+\campo\data"
path_dataout = path_data + r"\mails_matches\matched\confirmaciones_de_envio"
path_datain = path_data + r"\mails_matches\to_match"
path_deleted = path_data + r"\mails_matches\deleted"

In [3]:
#Importo bases de seleccion (tienen la variable nombre, n_adds_abv_mun y algunas otras)
files = tools.crea_lista_de_archivos(path_data + fr"\randomization\round2_2022_05_03", "dta")

df_concat = []
for file in files:
    df_temp = pd.read_stata(file)
    df_concat += [df_temp]
    
scrap_full = pd.concat(df_concat)
scrap_full = scrap_full.reset_index(drop=True)
scrap_full['script_string_3'] = scrap_full['script_string_3'].astype('str')

In [6]:
#Importar base de mails recibidos 
rtas_p1 = pd.read_excel(path_datain + fr"\mails_total_{fecha_descarga}_parte1.xlsx")
rtas_p2 = pd.read_excel(path_datain + fr"\mails_total_{fecha_descarga}_parte2.xlsx")
rtas_p3 = pd.read_excel(path_datain + fr"\mails_total_{fecha_descarga}_parte3.xlsx")

rtas_full = pd.concat([rtas_p1,rtas_p2,rtas_p3])
rtas_full.to_excel(path_datain + fr"\mails_total_{fecha_descarga}.xlsx")

In [7]:
# Base de correos desde los que enviamos
mails_df = pd.read_excel(path_dropbox + r"\BID-LGBTQ+\campo\0_instrucciones\2-Emails para Intervención.xlsx")
mails_df['Usuario_clean'] = mails_df.Usuario.apply(matching.clean_mail)
users = mails_df['Usuario_clean'].to_list()

In [8]:
#### Elimino observaciones que se corresponden a mails de prueba o de Google

# Dropeo observaciones que tienen Google en el mail / user
prev = len(rtas_full)
rtas_full_out = rtas_full[rtas_full['from'].str.contains("Google", na=False) == True]
rtas_full =     rtas_full[rtas_full['from'].str.contains("Google", na=False) == False]
print("Se dropearon", prev - len(rtas_full), "observaciones.")

# Dropeo observaciones de prueba de Juli
prev = len(rtas_full) 
to_concat = rtas_full[rtas_full['from'].str.contains("Julian test", na=False) == True]
rtas_full = rtas_full[rtas_full['from'].str.contains("Julian test", na=False) == False]
rtas_full_out = pd.concat([rtas_full_out,rtas_full[rtas_full['from'].str.contains("Julian test", na=False) == True]])

print("Se dropearon", prev - len(rtas_full), "observaciones.")

# Dropeo observaciones de prueba
prev = len(rtas_full) 
to_concat = rtas_full[rtas_full['subject'].str.contains("prueba", na=False) == True]
rtas_full = rtas_full[rtas_full['subject'].str.contains("prueba", na=False) == False]
rtas_full_out = pd.concat([rtas_full_out,to_concat])
print("Se dropearon", prev - len(rtas_full), "observaciones.")

# Dropeo observaciones de prueba (from == to, ¡nos los autoenviamos!)
prev = len(rtas_full) 
for user in users:
    to_concat = rtas_full[rtas_full['from'].apply(matching.clean_mail).str.contains(user)==True ]
    rtas_full = rtas_full[rtas_full['from'].apply(matching.clean_mail).str.contains(user)==False]
    rtas_full_out = pd.concat([rtas_full_out,to_concat])
print("Se dropearon", prev - len(rtas_full), "observaciones.")



# Exporto las obs dropeadas
rtas_full_out.to_excel(path_deleted + r"\observaciones_eliminadas_round_{ronda}_{fecha_out}.xlsx")

Se dropearon 423 observaciones.
Se dropearon 221 observaciones.
Se dropearon 146 observaciones.
Se dropearon 656 observaciones.


In [9]:
##### Identifica mails que vienen de properati 'contacts@mailing.properati.com'

# Filtro nulls
before = len(rtas_full)
rtas_full = rtas_full.dropna(subset=['from','body'])
after = len(rtas_full)
print("Se dropearon", before-after, "observaciones.")

# Creo binaria con True para los mails de properati
mails_properati = ['contacts@mailing.properati.com']

rtas_full['conf_envio'] = rtas_full['from'].apply(matching.match_string_con_lista, lista_matchs=mails_properati)
print("Match confirmaciones realizado.\nCantidad de mails de @properati:")

# Reporta los resultados
rtas_full['conf_envio'].value_counts()

Se dropearon 20 observaciones.
Match confirmaciones realizado.
Cantidad de mails de @properati:


True     7281
False    4931
Name: conf_envio, dtype: int64

In [10]:
##### Limpio las fechas
rtas_full['date_clean'] = rtas_full['date'].str.replace(".+,\s","",regex=True)
rtas_full['date_clean'] = rtas_full['date_clean'].str.replace("\-|\+|(0\d00)(.*)","",regex=True)
rtas_full['date_clean'] = rtas_full['date_clean'].str.replace("GMT","")
rtas_full['date_clean'] = rtas_full['date_clean'].str.lstrip(' ').str.rstrip(' ')
rtas_full['date_clean'] = rtas_full['date_clean'].str.zfill(20)
rtas_full['date_clean'] = pd.to_datetime(rtas_full['date_clean'], format="%d %b %Y %H:%M:%S")
rtas_full['date_clean'] = rtas_full['date_clean'].dt.date  # Solo me quedo dia-mes-año

In [10]:
##### Matcheo @properati con scraps
from importlib import reload
reload(matching)
rtas_full.reset_index(drop=True, inplace=True)

fin_ronda_1 = pd.to_datetime("2022-05-03").date()
rtas_properati = rtas_full.loc[(rtas_full['date_clean']>fin_ronda_1) & (rtas_full['conf_envio']==True)]

today = datetime.now()
today = today.strftime("%Y_%m_%d")
nombre = f'mails_conf_envio_round_{ronda}_{today}'

rtas_properati['index_scrap'] = np.nan
rtas_properati['match_confiab'] = np.nan
rtas_properati['match_step'] = np.nan
rtas_properati['match_string'] = np.nan
rtas_properati['match2_string'] = np.nan
scrap_full['index_rtas'] = np.nan
scrap_full['match_confiab'] = np.nan
scrap_full['match_step'] = np.nan

matched_arg = matching.full_proceso_solo_id(path=path_dataout, rtas_full=rtas_properati[rtas_properati['pais']=='arg'], scrap_full=scrap_full[scrap_full['pais']=='arg'], pais='_ARG')
matched_col = matching.full_proceso_solo_id(path=path_dataout, rtas_full=rtas_properati[rtas_properati['pais']=='col'], scrap_full=scrap_full[scrap_full['pais']=='col'], pais='_COL')
matched_ecu = matching.full_proceso_solo_id(path=path_dataout, rtas_full=rtas_properati[rtas_properati['pais']=='ecu'], scrap_full=scrap_full[scrap_full['pais']=='ecu'], pais='_ECU')
matched_per = matching.full_proceso_solo_id(path=path_dataout, rtas_full=rtas_properati[rtas_properati['pais']=='per'], scrap_full=scrap_full[scrap_full['pais']=='per'], pais='_PER')

# Exporto los matcheados a un único Excel
matched = pd.concat([matched_arg,matched_col,matched_ecu,matched_per])
matched.to_excel(path_dataout + "\\sent_" + nombre +".xlsx")
print(path_dataout + "\\sent_" + nombre +".xlsx creado")

['pablom.ngomez@gmail.com' 'jcmanuelrodrigue.z@gmail.com'
 'gome.zpablou@gmail.com' 'rodriguez.j.cmanuel@gmail.com'
 'pablom.ngome.z@gmail.com' 'rodriguez.manuel.j.f@gmail.com'
 'ro.driguezmanuelw@gmail.com' 'manuelrodrigue.zjf@gmail.com'
 'rodriguez.j.c.manuel@gmail.com' 'gomezf.hpablo@gmail.com'
 'f.h.pablogomez@gmail.com' 'rodriguez.manuelj.f@gmail.com'
 'p.hpablo.gomez@gmail.com' 'pablogome.z.f.h@gmail.com'
 'rodriguezj.cmanuel@gmail.com' 'pablo.gomez.f.h@gmail.com'
 'jfmanuelrodrigue.z@gmail.com' 'pablogome.zf.h@gmail.com'
 'j.fmanuelrodriguez@gmail.com' 'p.hpablogome.z@gmail.com'
 'j.fmanuel.rodriguez@gmail.com' 'f.h.pablo.gomez@gmail.com'
 'rodrigue.zmanuel.w@gmail.com' 'p.h.pablo.gomez@gmail.com'
 'rodrigue.z.manuel.w@gmail.com' 'rodrigue.zmanuelw@gmail.com'
 'rodriguezmanuelj.f@gmail.com' 'gomezf.h.pablo@gmail.com'
 'f.hpablogomez@gmail.com' 'j.cmanuelrodriguez@gmail.com'
 'manuel.rodriguez.j.f@gmail.com' 'p.h.pablogomez@gmail.com'
 'gomezfh.pa.blo@gmail.com' 'gome.zpablo.u@gm

In [13]:
matched._merge.value_counts()

both          3636
left_only        0
right_only       0
Name: _merge, dtype: int64

In [14]:
#### TODO:
# Verificar manualmente que estos estén bien. El id de properati es muy corto así que accidentalmente podría generar errores de matcheo.
matched[matched.id.str.len()<5]

,index_x,inmobiliaria,municipio,nombre,fecha,link,ciudad,pais_x,descripcion,precio,moneda,caracteristicas,ambientes,habitaciones,banos,m2_totales,m2_cubiertos,cocheras,id,precio_local,p_m2,p_m2_above_mdn,n_adds,n_adds_above_mdn,m2_above_mdn,common_name,capital,treatment,ses,male_name,mail,male_couple,female_couple,script,script_string,script_string_2,script_string_3,randomization_seed,nombre_2,profesional,vivienda,n_adds_muni,n_adds_muni_above_mdn,random,index_rtas,match_confiab_x,match_step_x,Unnamed: 0,subject,from,to,body,date,cc,thread_id,mimeType,pais_y,index_y,conf_envio,date_clean,index_scrap,match_confiab_y,match_step_y,match_string,match2_string,_merge
19,719,heredia,caba,departamento en alquiler en neuquen 602,2022-02-18,properati.com.ar/detalle/4ae45_alquiler_depart...,buenos aires ampliado,arg,"edificio de 14 pisos, 5 departamentos por piso...",32000.0,$,,NaN,NaN,NaN,NaN,NaN,,150,32000.0,NaN,NaN,5.0,1.0,NaN,2.0,1.0,trans,neutral,Manuel,rodriguez.j.cmanuel@gmail.com,Luis,Belén,2.0,"Hola, somos una pareja interesada en alquilar ...",Manuel y Belén (ella es trans),Código 150,2.0,NaN,NaN,NaN,NaN,NaN,NaN,19.0,100.0,1.0,1214,Hiciste una consulta por la propiedad Neuquen 602,Properati <contacts@mailing.properati.com.ar>,rodriguez.j.cmanuel@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --_x000D_\n_x00...,"Wed, 04 May 2022 17:25:48 +0000 (UTC)",NaN,180901b4bb14d042,multipart/alternative,arg,141.0,True,2022-05-04,16.0,100.0,1.0,"[Match(start=1351, end=1360, dist=0, matched='...",nan,both
2,1360,amadeo propiedades,lomas_de_zamora,departamento en alquiler en hipolito yrigoyen ...,2022-01-11,properati.com.ar/detalle/47suq_alquiler_depart...,buenos aires ampliado,arg,"departamento 2 amb, ubicado en hipolito yrigoy...",26000.0,$,"['2ambientes', '1bano', '26 m²totales']",2.0,NaN,1.0,26.0,NaN,,5389,26000.0,1000.000000,0.0,2.0,0.0,0.0,2.0,0.0,trans,neutral,Manuel,rodriguez.j.cmanuel@gmail.com,Pablo,Florencia,1.0,"Buenos días, mi nombre es Manuel, junto a mi p...",Manuel y Florencia.,Código 5389,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,100.0,1.0,1149,Hiciste una consulta por la propiedad Hipolito...,Properati <contacts@mailing.properati.com.ar>,rodriguez.j.cmanuel@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --_x000D_\n_x00...,"Wed, 04 May 2022 19:52:48 +0000 (UTC)",NaN,18090a1dda3ef7c7,multipart/alternative,arg,76.0,True,2022-05-04,24.0,100.0,1.0,"[Match(start=1533, end=1543, dist=0, matched='...",nan,both
10,943,raices puerto madero,caba,departamento en alquiler en florida 1065,2021-09-12,properati.com.ar/detalle/3z6bc_alquiler_depart...,buenos aires ampliado,arg,se trata de un departamento en el edificio kav...,900.0,usd,,NaN,NaN,NaN,NaN,NaN,,1373,179550.0,NaN,NaN,1.0,0.0,NaN,1.0,1.0,homo,neutral,Pablo,pablom.ngome.z@gmail.com,Manuel,,2.0,"Hola, somos una pareja interesada en alquilar ...",Pablo y Manuel,Código 1373,2.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,100.0,1.0,1539,Hiciste una consulta por la propiedad Florida ...,Properati <contacts@mailing.properati.com.ar>,pablom.ngome.z@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Wed, 04 May 2022 17:43:55 +0000 (UTC)",NaN,180902be574364f4,multipart/alternative,arg,NaN,True,2022-05-04,18.0,100.0,1.0,"[Match(start=1276, end=1286, dist=0, matched='...",nan,both
2,1415,paula di gaeta estudio inmobiliario,lomas_de_zamora,departamento en alquiler en fray mamerto esqui...,2022-04-26,properati.com.ar/detalle/4g9js_alquiler_depart...,buenos aires ampliado,arg,departamento de dos ambientes opcional con coc...,22000.0,$,"['2ambientes', '1bano', '30 m²totales']",2.0,NaN,1.0,30.0,NaN,,3891,22000.0,733.333313,0.0,8.0,1.0,0.0,2.0,0.0,trans,neutral,Pablo,pablogome.zf.h@gmail.com,Luis,Laura,1.0,"Buenos días, mi nombre es Pablo, junto a mi pa...",Pablo y Laura.,Código 3891,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,100.0,1.0,42,Hiciste una consulta por la propiedad Fray Mam...,Properati <contacts@mailing.properati.com.ar>,pablogome.zf.h@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Wed, 04 May 2022 20:09:32 +0